In [1]:
from dotenv import load_dotenv
from syncflowai import store_processed_mission, store_raw_response, store_feedback, generate_feedback, extract_mission_details, generate_mission
import datetime
import json
import openai
load_dotenv()

True

In [18]:
current_date = datetime.datetime.now() + datetime.timedelta(days=10)
mission_count = 0
total_days = 15  # Durée totale en jours
missions_per_day = 7  # Missions par jour

for i in range(total_days * missions_per_day):
    print(f"Mission {i+1}", end="\r")
    mission = generate_mission()
    print(f"Extracting details for mission {i+1}")
    _, raw_response = extract_mission_details(mission)
    
    # Calcul de la limite inférieure de la note en fonction du jour
    day_number = (current_date - (datetime.datetime.now() + datetime.timedelta(days=10))).days
    lower_rating_limit = min(5, 1 + day_number // (total_days // 4))
    
    print(f"Generating feedback for mission {i+1}")
    feedback = generate_feedback(mission = mission, raw_response = raw_response, current_date = current_date, rating_limits=[lower_rating_limit, 5])
    print(lower_rating_limit)
    print(f"Storing data for mission {i+1}")
    store_raw_response(raw_response)
    store_feedback(feedback)

    mission_count += 1
    if mission_count >= missions_per_day:
        current_date += datetime.timedelta(days=1)
        mission_count = 0
        print(current_date)

KeyboardInterrupt: 

In [9]:
mission = generate_mission()

In [10]:
mission

'I am seeking a developer or a team of developers to work on a project that requires proficiency in either Java or C++. The task involves developing a specific application or feature (please insert specific application or feature details here). The project should be completed within a set timeframe (please provide a timeframe), adhering to the agreed-upon specifications and quality standards.\n\nThe developer(s) should be able to demonstrate:\n\n1. Strong knowledge in the selected programming language (Java or C++), including the ability to write clean, efficient, and well-documented code.\n2. Experience with software development best practices and design patterns relevant to the task at hand.\n3. The ability to troubleshoot, debug, and optimize code to improve performance and reliability.\n4. Quality testing and validation to ensure the end product meets the defined requirements.\n5. Clear and regular communication regarding project progress, challenges, and deadlines.\n\nProject prog

In [11]:
details = extract_mission_details(mission)

In [12]:
details[0]

{'name': 'Custom Software Development Project',
 'abstract': 'Development of a specific application or feature using Java or C++, to be completed within a given timeframe adhering to high-quality standards.',
 'detail': 'We are looking for a developer or a team of developers skilled in <b>Java or C++</b> to undertake the development of a certain application or feature. The specifics of the application or feature need to be stated to tailor the expertise required. The project must be completed within the stipulated timeframe which should be defined. <br></br>Key requirements for the developer(s) include:<br></br>1. Proficiency in <b>Java or C++</b>, showcasing the ability to craft clean, efficient, and well-documented code.<br></br>2. Familiarity with <b>software development best practices and design patterns</b> for the particular task.<br></br>3. Competence in <b>troubleshooting, debugging, and code optimization</b> for better performance and reliability.<br></br>4. Commitment to <b>q

In [5]:
import MySQLdb

ModuleNotFoundError: No module named 'mysqlclient'

In [7]:
mysqlclient.__version__

NameError: name 'mysqlclient' is not defined